In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator


In [7]:
def als_recommender(filename, seed):
    '''
    This function prints the RMSE of recommendations obtained
    through ALS collaborative filtering using 'BX-Book-Ratings.csv' dataset.
    The training ratio is 80% and the test ratio is 20%. The
    random seed is used for both sampling and ALS optimizer initialization.
    '''

    # Initialize Spark session
    spark = SparkSession.builder.appName("ALSRecommender").getOrCreate()

    # Load data from a CSV file, considering semicolon delimiter and quotes
    data = spark.read.csv(filename, header=True, inferSchema=True, sep=';', quote='"')

    # Select and rename the columns according to the CSV file's format
    ratings = data.select(
        col('User-ID').cast('int').alias('userId'), 
        col('ISBN').alias('bookId'), 
        col('Book-Rating').cast('int').alias('rating')
    )

    # Transform the ISBN string to an index using StringIndexer
    stringIndexer = StringIndexer(inputCol="bookId", outputCol="bookIdIndexed")
    model = stringIndexer.fit(ratings)
    ratingsIndexed = model.transform(ratings)

    # Split data into training and test sets
    (training, test) = ratingsIndexed.randomSplit([0.8, 0.2], seed=seed)


    # Build the recommendation model using ALS on the training data
    """als = ALS(maxIter=5, rank=70, regParam=0.01, userCol="userId", 
              itemCol="bookIdIndexed", ratingCol="rating", coldStartStrategy="drop", 
              seed=seed)
    """
    als = ALS(userCol="userId", 
              itemCol="bookIdIndexed", ratingCol="rating", coldStartStrategy="drop", 
              seed=seed)
        
    #Creating a Parameter Grid for grid search
    paramGrid = ParamGridBuilder() \
        .addGrid(als.maxIter, [5, 7]) \
        .addGrid(als.rank, [70, 80]) \
        .addGrid(als.regParam, [0.01, 0.1]) \
        .build()
    
    #model = als.fit(training)
   
    # Make predictions on the test data
    #predictions = model.transform(test)

    # Evaluate the model by computing the RMSE on the test data
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", 
                                    predictionCol="prediction")
    
    cv = CrossValidator(estimator=als, estimatorParamMaps=paramGrid,
                        evaluator=evaluator,
                        numFolds=2)
    model = cv.fit(training)
    predictions = model.transform(test)
    rmse = evaluator.evaluate(predictions)
    print("Root-mean-square error = " + str(rmse))

    # Stop the Spark session
    spark.stop()

    return rmse



In [8]:
filename = 'Book reviews/BX-Book-Ratings.csv'
seed = 12345
rmse = als_recommender(filename, seed)

Py4JJavaError: An error occurred while calling o5024.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:2